## 바텀 라인의 영향력 분석

In [1]:
import pandas as pd
import requests
from urllib.parse import quote as quotekr # 한글 아이디를 인코딩 parse.quote
import json
import time

In [2]:
api_key = 'RGAPI-6eb393c4-ed4a-45ba-86dd-d23322be4d36'
request_header = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                "Origin": "https://developer.riotgames.com",
                "X-Riot-Token": api_key
                }

#### 티어별 유저 5명 선별
* op.gg에서 무작위 추출
* Most top 3 챔피언이 모두 원거리 딜러
* 랭크 선호 포지션이 바텀 비율이 월등히 높은 사람
* 솔로랭크 50판 이상

In [3]:
grand_master_user = ['줄라이77', '바텀개', 'always crying', '침대위섹시카이팅', '미드KING']
diamond_user = ['세혀닝', '강한남자한만윤', 'FEARN0T', '잘해봐요괜찮아요', '쿠로미짱이현']
emerald_user = ['정의는 승리한다v', 'no return0', 'Cigar3', '주희보고싶은민교', 'Asnm']
platinum_user = ['협곡의 맙소사', '엄딴식', '지구뚝배기', '08년생 큐티정민', 'Gucci Vs']
gold_user = ['루픠는못말려', '홍구형', '카이팅의 귀재', 'Orangerie', '쥐파티']

grand_master_puuid = []
diamond_puuid = []
emerald_puuid = []
platinum_puuid = []
gold_puuid = []

In [ ]:
def user_puuid(users, puuids):
    for user in users:
        url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
        response = requests.get(url, headers=request_header)
        
        if response.status_code == 200: # Success
            pass

        elif response.status_code == 429: # Rate limit exceeded
            print('Rate limit exceeded error, api cost full')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 3 second wait time')
                    time.sleep(3)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 503: # Service unavailable
            print('Service unavailable error')
            start_time = time.time()

            while True:
                if response.status_code == 200:
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                else:
                    print('try 5 second wait time')
                    time.sleep(5)
                    response = requests.get(url, headers=request_header)

        elif response.status_code == 403: # Forbidden
            print('Forbidden error, you need api renewal')
            print('break')
            return

        puuids.append(response.json()['puuid'])

In [5]:
# def user_puuid2(users, puuids):
#     for user in users:
#         url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + quotekr(user)
#         response = requests.get(url, headers=request_header)

#         if response.status_code == 200:
#             pass
        
#         elif response.status_code == 429:
#             print("sleep", user, ' ', response.headers['Retry-After'])
#             time.sleep(int(response.headers['Retry-After']) + 5)
#             response = requests.get(url, headers=request_header)

#         puuids.append(response.json()['puuid'])

In [ ]:
user_puuid(grand_master_user, grand_master_puuid)
user_puuid(diamond_user, diamond_puuid)
user_puuid(emerald_user, emerald_puuid)
user_puuid(platinum_user, platinum_puuid)
user_puuid(gold_user, gold_puuid)

In [ ]:
grand_master_df = pd.DataFrame({
    'tier': ['grand_master' for i in range(5)],
    'summoner': grand_master_user,
    'puuid': grand_master_puuid
    })

diamond_df = pd.DataFrame({
    'tier': ['diamond' for i in range(5)],
    'summoner': diamond_user,
    'puuid': diamond_puuid
    })

emerald_df = pd.DataFrame({
    'tier': ['emerald' for i in range(5)],
    'summoner': emerald_user,
    'puuid': emerald_puuid
    })

platinum_df = pd.DataFrame({
    'tier': ['platinum' for i in range(5)],
    'summoner': platinum_user,
    'puuid': platinum_puuid
    })

gold_df = pd.DataFrame({
    'tier': ['gold' for i in range(5)],
    'summoner': gold_user,
    'puuid': gold_puuid
    })

In [ ]:
df = pd.concat([grand_master_df, diamond_df, emerald_df, platinum_df, gold_df], ignore_index=True)
display(df.head(2))
display(df.tail(2))

In [76]:
df.to_excel('puuid_data.xlsx', index=False)

In [77]:
df = pd.read_excel('puuid_data.xlsx')
df.head(2)

,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


#### 데이터 수집
* '바텀 게임' 이라는 말이 줄기 시작한 13.6패치가 3월 중순
* 1주일에 랭크게임 5판 한다고 가정
* 현재 8월 말이므로 약 20주
* 100게임 이전 인당 10판

In [79]:
# 소환사 협곡 : response.json()['info']['gameMode'] == 'CLASSIC'
# 랭크 게임 : len(response.json()['info']['teams'][0]['bans']) == 0 (자유랭크도 포함됨)

def puuid_to_matchid(puuid):
    match_ids = []
    start = 110

    while len(match_ids) < 10: # 10경기 추출
        matches_url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start}&count=20'
        match_list_resp = requests.get(matches_url, headers=request_header)

        if match_list_resp.status_code == 429: # Rate limit exceed
            print("sleep match_list", match_list_resp.headers['Retry-After'])
            time.sleep(int(match_list_resp.headers['Retry-After']) + 5)
            match_list_resp = requests.get(matches_url, headers=request_header)

        if match_list_resp.status_code == 200:
            for mat_id in match_list_resp.json():
                if len(match_ids) == 10:
                    break

                mat_url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mat_id
                mat_data_resp = requests.get(mat_url, headers=request_header)

                if mat_data_resp.status_code == 429: # Rate limit exceed
                    print("sleep mat_data", mat_data_resp.headers['Retry-After'])
                    time.sleep(int(mat_data_resp.headers['Retry-After']) + 5)
                    mat_data_resp = requests.get(mat_url, headers=request_header)

                if mat_data_resp.status_code == 200:
                    if mat_data_resp.json()['info']['gameMode'] == 'CLASSIC' \
                        and len(mat_data_resp.json()['info']['teams'][0]['bans']) > 0 \
                        and mat_data_resp.json()['info']['participants'][0]['gameEndedInEarlySurrender'] == False:
                        # 소환사 협곡이고 밴이 있는 경기 -> 랭크 게임
                        # 재시작 경기 x
                        match_ids.append(mat_id)
      
            start += 20

    return match_ids

In [73]:
col = [('match' + str(x)) for x in range(1, 11)]
df_puuid = pd.DataFrame(columns=col)
df_puuid

,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10


In [78]:
display(df.head(2))
display(df.tail(2))

,tier,summoner,puuid
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...


,tier,summoner,puuid
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...


In [80]:
for puuid in df['puuid']:
    matchid_list = puuid_to_matchid(puuid)
    df_mid = pd.DataFrame([matchid_list[::-1]], columns=col)
    df_puuid = pd.concat([df_puuid, df_mid], ignore_index=True)

sleep mat_data 90
sleep mat_data 90
sleep mat_data 89


In [81]:
display(df_puuid.head(2))
display(df_puuid.tail(2))

,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,KR_6623745063,KR_6623836624,KR_6623904704,KR_6623973606,KR_6624290079,KR_6624379039,KR_6624442133,KR_6624480871,KR_6625973763,KR_6626038023
1,KR_6663860934,KR_6663918544,KR_6663968631,KR_6664044375,KR_6666840236,KR_6666888820,KR_6667062688,KR_6667098846,KR_6667214060,KR_6667275166


,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
23,KR_6650763917,KR_6652443964,KR_6652486937,KR_6652755462,KR_6652770663,KR_6652784956,KR_6652811546,KR_6652854733,KR_6652894448,KR_6652932830
24,KR_6525391402,KR_6531519533,KR_6531625161,KR_6531672863,KR_6532015594,KR_6532031969,KR_6532087238,KR_6532107034,KR_6532125986,KR_6532620540


In [82]:
df2 = pd.concat([df, df_puuid], axis=1)
display(df2.head(2))
display(df2.tail(2))

,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6623745063,KR_6623836624,KR_6623904704,KR_6623973606,KR_6624290079,KR_6624379039,KR_6624442133,KR_6624480871,KR_6625973763,KR_6626038023
1,grand_master,바텀개,EXr7XpD4deA3djszdNHUGUOVNcRxMwC26KdDhgGE3fJk9H...,KR_6663860934,KR_6663918544,KR_6663968631,KR_6664044375,KR_6666840236,KR_6666888820,KR_6667062688,KR_6667098846,KR_6667214060,KR_6667275166


,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
23,gold,Orangerie,lAoGAWR0v4y0QbDTOMFKjFv_LlKfh_ymsZYyHsLkrr8xGM...,KR_6650763917,KR_6652443964,KR_6652486937,KR_6652755462,KR_6652770663,KR_6652784956,KR_6652811546,KR_6652854733,KR_6652894448,KR_6652932830
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...,KR_6525391402,KR_6531519533,KR_6531625161,KR_6531672863,KR_6532015594,KR_6532031969,KR_6532087238,KR_6532107034,KR_6532125986,KR_6532620540


In [83]:
df2.to_excel('matchid_data.xlsx', index=False)

In [84]:
df2 = pd.read_excel('matchid_data.xlsx')

In [85]:
display(df2.loc[[0]])
display(df2.loc[[24]])

,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
0,grand_master,줄라이77,ZPFTEsnEwgz0EfXyXbdlj3HIzoKdjLJmkM4p8v4H9EYlEW...,KR_6623745063,KR_6623836624,KR_6623904704,KR_6623973606,KR_6624290079,KR_6624379039,KR_6624442133,KR_6624480871,KR_6625973763,KR_6626038023


,tier,summoner,puuid,match1,match2,match3,match4,match5,match6,match7,match8,match9,match10
24,gold,쥐파티,CFbEyz3fMIYwR8t9htV_iKMT0OZKNK6gw0KWLeTajCG1aw...,KR_6525391402,KR_6531519533,KR_6531625161,KR_6531672863,KR_6532015594,KR_6532031969,KR_6532087238,KR_6532107034,KR_6532125986,KR_6532620540


#### 경기 데이터 전처리

In [42]:
def make_base_dict(puuid, matchid):
    base_dict = {'puuid': puuid, 'matchid': matchid}
    
    return base_dict

In [43]:
def response_match(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid
    resp_ma = requests.get(url, headers=request_header)

    return resp_ma

In [44]:
# participant ID 찾기
### match 에서는 pid-1
### timeline 에서는 'pid'
def find_participantId(resp_ma, puuid):
    return resp_ma.json()['metadata']['participants'].index(puuid) + 1

In [45]:
def make_data_end_dict(resp_ma, parti_id):
    data_end = resp_ma.json()['info']['participants'][parti_id - 1]

    data_end_dict = {'win': data_end['win'],
                     'kills': data_end['kills'],
                     'deaths': data_end['deaths'],
                     'assists': data_end['assists'],
                     'bountyGold': data_end['challenges']['bountyGold'],
                     'turretKills': data_end['turretKills'],
                     'turretsLost': data_end['turretsLost']
                    }
    
    return data_end_dict

In [46]:
def response_timelines(matchid):
    url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + matchid + '/timeline'
    resp_tl = requests.get(url, headers=request_header)
    
    return resp_tl

In [47]:
def make_data_15_dict(resp_tl, parti_id):
    data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(parti_id)]

    data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                    'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                    'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                    'level15': data_15['level'],
                    'xp15': data_15['xp'],
                    'minionsKilled15': data_15['minionsKilled'],
                    'totalGold15': data_15['totalGold']
                    }
    
    return data_15_dict

In [48]:
def make_data_15_kda_dict(resp_tl, parti_id):
    data_15_kda_dict = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

    for times in range(0, 16):
        events = resp_tl.json()['info']['frames'][times]['events']

        for i in range(len(events)):
            if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
                
                if parti_id == events[i]['killerId']:
                    data_15_kda_dict['kills15'] += 1

                if parti_id == events[i]['victimId']:
                    data_15_kda_dict['deaths15'] += 1

                if parti_id in events[i]['assistingParticipantIds']:
                    data_15_kda_dict['assists15'] += 1

    return data_15_kda_dict

In [49]:
match_cols = ['puuid', 'matchid', \
              'win', 'kills', 'deaths', 'assists', 'bountyGold', 'turretKills', 'turretsLost', \
              'totalDamageDone15', 'totalDamageDoneToChampions15', 'totalDamageTaken15', 'level15', 'xp15', 'minionsKilled15', 'totalGold15', \
              'kills15', 'deaths15', 'assists15']

In [65]:
df_match = pd.DataFrame(columns=match_cols)
df_match

,puuid,matchid,win,kills,deaths,assists,bountyGold,turretKills,turretsLost,totalDamageDone15,totalDamageDoneToChampions15,totalDamageTaken15,level15,xp15,minionsKilled15,totalGold15,kills15,deaths15,assists15


In [66]:
for i in range(len(df2)):
    puuid = list(df2.loc[[i]]['puuid'])[0]
    match_ids = list(df2.loc[i])[3:]
    
    for matchid in match_ids:
        base_dict = make_base_dict(puuid, matchid)
        
        resp_ma = response_match(matchid)
        if resp_ma.status_code == 429: # Rate limit exceed
            print("sleep(ma)", resp_ma.headers['Retry-After'])
            time.sleep(int(resp_ma.headers['Retry-After']) + 5)
            resp_ma = response_match(matchid)

        parti_id = find_participantId(resp_ma, puuid)
        data_end_dict = make_data_end_dict(resp_ma, parti_id)

        resp_tl = response_timelines(matchid)
        if resp_tl.status_code == 429: # Rate limit exceed
            print("sleep(tl)", resp_tl.headers['Retry-After'])
            time.sleep(int(resp_tl.headers['Retry-After']) + 5)
            resp_tl = response_match(matchid)

        data_15_dict = make_data_15_dict(resp_tl, parti_id)
        data_15_kda_dict = make_data_15_kda_dict(resp_tl, parti_id)

        base_dict.update(data_end_dict)
        base_dict.update(data_15_dict)
        base_dict.update(data_15_kda_dict)
        df_match = df_match.append(base_dict, ignore_index=True)

    time.sleep(5)

IndexError: list index out of range

In [67]:
resp_tl.json()

{'metadata': {'dataVersion': '2',
  'matchId': 'KR_6544411741',
  'participants': ['_IJOLUFK_yHR5aoFa-IswJ7ZT1iSv6QK9ao5UoW6hIIAsxoI8FAzv9qrP70UQrniHAqwgZJHvRw0cA',
   'pgqJLAKxByu4VyP414F4GPi2KxaQxCqD5vkdW87nQm_Hrl-oIL35s8LLYpeTKh2b-LSbftSlPxjUmA',
   '3aSCreBL5eJzzRbcA4ZpSKt8-FfLWupQW4HmYqjdOUTW2LdR1F8_I7HZLMExgg28-dt3peEfNYtWjQ',
   'uV5qnid1e3yONnBWKQ3-Gg19xEhSMDR-eoj2FKAQkMMP3dAlqjK1qbXtyiWfq9tdQNDoG0ltyFtNwg',
   'WbjRJLa-Zx6VTcc1zWhmZtYt4YPL93MD9x-GwT7a7ezpH25Y0eu_ZCUA1nTTNN30xEOsVD2Rz6lDPQ',
   'RbQ-y9t8qLVLQ3jqljp0m24F60qgQ-D8ktgFBwVrUn4jNP4xruatiJXhATBgILvMw_OwAgjD1Y8cPg',
   'b2LjIrQJazQv5ZwM9zlvnvTci_ECHVnDUqF9DxSFOOYjW5N4x0qqpUPTUFY-iJfWdE25hASb88tx9Q',
   'VTSircnf6-cEHxTXYHppPvHu1Wkkd_-fIFbPGSQghuNEZwFuBcumCA74YTrSohqRTk3gTkqQ9D8ang',
   '4ZSpLbeybSbDG-s52ZQSsxQdGvtQbKZdvf_B-UwZ3P1_DJfm-h7HxASxEhaF7A1qI5IS-iPpW2d9Yw',
   'K3soLA_W8AU49z5zXnRBQzgrAgLfIaIzXxRCVU2BTx30gH7jZwlXYp1w-EQ2kkC5kKz289ZeBm99sQ']},
 'info': {'frameInterval': 60000,
  'frames': [{'events': [{'realTim

In [ ]:
display(df_match.head(2))
display(df_match.tail(2))

#### 연습구간

In [ ]:
# parti_ID 찾기
def find_participantId(response, puuid):
    return response.json()['metadata']['participants'].index(puuid) + 1

In [ ]:
find_participantId(resp_tl, df2['puuid'][0])

In [ ]:
resp_tl.json()['info']['frames'][15]['participantFrames']['9']

In [ ]:
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDone']) # 15분까지 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageDoneToChampions']) # 15분까지 챔피언에게 가한 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['damageStats']['totalDamageTaken']) # 15분까지 받은 피해량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['level']) # 15분 레벨
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['minionsKilled']) # 15분 미니언 처치 수
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['totalGold']) # 15분 골드 누적 획득량
print(resp_tl.json()['info']['frames'][15]['participantFrames']['9']['xp']) # 15분 경험치 누적 획득량

# 게임 종료
print(response.json()['info']['participants'][8]['challenges']['bountyGold']) # 현상금
print(response.json()['info']['participants'][8]['kills']) # 킬
print(response.json()['info']['participants'][8]['deaths']) # 데스
print(response.json()['info']['participants'][8]['assists']) # 어시스트
print(response.json()['info']['participants'][8]['turretKills']) # 자신이 파괴한 포탑 수
print(response.json()['info']['participants'][8]['turretsLost']) # 전체 파괴된 포탑 수
print(response.json()['info']['participants'][8]['win']) # 승패 여부

In [ ]:
resp_tl.json()['info']['frames'][8]['events'][22].keys()

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

##### 테스트

In [ ]:
pud = df2['puuid'][0]
mid = df2['match1'][0]
basedict = {'puuid': pud, 'matchid': mid}

In [ ]:
url1 = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid
resp_end = requests.get(url1, headers=request_header)
part_id = find_participantId(resp_end, pud)

In [ ]:
part_id

In [ ]:
data_end = resp_end.json()['info']['participants'][part_id-1]
data_end_dict = {'win': data_end['win'],
                 'kills': data_end['kills'],
                 'deaths': data_end['deaths'],
                 'assists': data_end['assists'],
                 'bountyGold': data_end['challenges']['bountyGold'],
                 'turretKills': data_end['turretKills'],
                 'turretsLost': data_end['turretsLost']
                 }

In [ ]:
data_end_dict

In [ ]:
url_tl = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + mid + '/timeline'
resp_tl = requests.get(url_tl, headers=request_header)

In [ ]:
data_15 = resp_tl.json()['info']['frames'][15]['participantFrames'][str(part_id)]
data_15_dict = {'totalDamageDone15': data_15['damageStats']['totalDamageDone'],
                'totalDamageDoneToChampions15': data_15['damageStats']['totalDamageDoneToChampions'],
                'totalDamageTaken15': data_15['damageStats']['totalDamageTaken'],
                'level15': data_15['level'],
                'xp15': data_15['xp'],
                'minionsKilled15': data_15['minionsKilled'],
                'totalGold15': data_15['totalGold']
                }

In [ ]:
data_15_dict

In [ ]:
x = resp_tl.json()['info']['frames'][8]['events']
for i in range(len(x)):
    if list(x[i])[0] == 'assistingParticipantIds':
        print(i, x[i]['assistingParticipantIds'], x[i]['killerId'], x[i]['victimId'])

        if 9 in x[i]['assistingParticipantIds']:
            print('\tassist')
        if 9 == x[i]['killerId']:
            print('\tkill')
        if 9 == x[i]['victimId']:
            print('\tdeath')

In [ ]:
data_15_kda = {'kills15': 0, 'deaths15': 0, 'assists15': 0}

In [ ]:
for times in range(0, 16):
    events = resp_tl.json()['info']['frames'][times]['events']

    for i in range(len(events)):
        if list(events[i])[0] == 'assistingParticipantIds' and events[i]['type'] == 'CHAMPION_KILL':
            
            if part_id == events[i]['killerId']:
                data_15_kda['kills15'] += 1

            if part_id == events[i]['victimId']:
                data_15_kda['deaths15'] += 1

            if part_id in events[i]['assistingParticipantIds']:
                data_15_kda['assists15'] += 1

In [ ]:
data_15_kda

In [ ]:
basedict.update(data_end_dict)
basedict.update(data_15_dict)
basedict.update(data_15_kda)

In [ ]:
basedict

In [ ]:
df_match.append(basedict, ignore_index=True)

##### -----

In [ ]:
tempdf = df_match.copy()
tempdf


In [ ]:
temp_dict = {'win':True, 'kills':10, 'deaths':20, 'assists':30}
td2 = {'new': False}

In [ ]:
tempdf = pd.DataFrame(columns=['win', 'kills', 'deaths', 'assists', 'new'])

In [ ]:
for key, value in temp_dict.items():
    print(key, value)
    tempdf.append(temp_dict, ignore_index=True)

tempdf

In [ ]:
temp_dict.update(td2)

In [ ]:
temp_dict

In [ ]:
tempdf.append(temp_dict, ignore_index=True)

In [ ]:
requests.get('http://ddragon.leagueoflegends.com/cdn/13.17.1/data/en_US/item.json').json()['data']['1001']['image']['full']

#### 연습구간 종료

In [ ]:
url = 'https://asia.api.riotgames.com/lol/match/v5/matches/' + df2['match1'][0] + '/timeline'
response = requests.get(url, headers=request_header)
printjs(response.json())

In [ ]:
response.json()['info'].keys()

In [ ]:
response.json()['info']['frames']

In [ ]:
temp = response.json()['info']
len(temp['frames'])

In [ ]:
for x in temp['frames']:
    del x['participantFrames']

In [ ]:
temp['frames']

#### 선택
KDA, bountyLevel, champLevel, goldEarned, timePlayed, totalDamageDealt, totalDamageTaken, win

#### 15분
* 서렌 가능 시간
* 포탑 방패가 사라지는 14분 + 이후 타워가 깨질 수 있는 시간 약 1분
* 챔피언 성능이 달라지는 1코어 이상(1.5코어) 아이템 소지

#### 10분?
* 다른 라이너의 큰 개입 없이 라인전이 이루어지는 시간
* 서포터까지도 충분히 궁극기를 배우고 교전에 적극 참여하기 충분한 시간
* 12~13분이면 라인전이 끝남, 15분은 거의 3번째 용